In [15]:
!pip install mistralai

  Using cached mistralai-0.1.8-py3-none-any.whl.metadata (1.8 kB)
  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached orjson-3.10.3-cp311-none-win_amd64.whl.metadata (50 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.18.2-cp311-none-win_amd64.whl.metadata (6.7 kB)
Using cached mistralai-0.1.8-py3-none-any.whl (15 kB)
Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached orjson-3.10.3-cp311-none-win_amd64.whl (138 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
Using cached pydantic_core-2.18.2-cp311-none-win_amd64.whl (1.9 MB)
Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
  

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.7.1 which is incompatible.


In [86]:
import pandas as pd
from fun import *

In [87]:
df = pd.read_csv("./translations.csv")
df.head()

,lp,src,mt,ref,score,raw,annotators,domain,year,l1,l2
0,en-zh,Police said in a statement at the time that th...,警方在当时的一份声明中表示，他们决定以“沉重的心情”结束对 Jacsun 的搜索，但指出他们...,警方在当时的一份声明中表示，他们在决定结束对杰森的搜寻时，是怀着“沉重的心情”的，并指出他们...,0.238989,80.500000,4,news,2020,en,zh
1,en-zh,"Trump will likely veto the resolution, the six...",特 朗 普 可 能 会 否 决 该 决 议 ， 这 是 他 第 六 次 以 总 统 身 份 ...,特朗普很可能行使总统否决权，这将是他成为总统以来第6次否决国会立法。他早在三月已动用否决权，...,0.487842,81.166667,6,news,2020,en,zh
2,en-zh,The man arrived at work about 9.30am after fai...,该名男子未能回复同事的几条短信和电话，于上午9.30左右上班。,该男子大约在上午 9:30 到达工作单位，在此之前，他未能回复同事发来的几条信息和打来的电话。,0.469850,80.500000,4,news,2020,en,zh
3,en-zh,Are hopes for a nuclear-free world realistic?,对一个无核世界的希望是现实的吗 ？,实现无核化世界的希望是否现实？,0.626218,81.000000,4,news,2020,en,zh
4,en-zh,The California attorney general's office in Ma...,"3月,加利福尼亚总检察长办公室在经过近一年的调查后拒绝提出州刑事指控,当时,总检察长Xavi...",3 月，加州总检察长办公室在完成了近一年的案件调查后，拒绝对两名警察提起加州刑事起诉。总检察...,-0.207602,59.000000,4,news,2020,en,zh


In [88]:
with open("./prompts/PROMPT3.txt", 'r') as f:
    prompt_template = f.read()

print(prompt_template)

Please evaluate the quality of the translation from [ORIGINAL LANGUAGE] to [TRANSLATED LANGUAGE] based on accuracy, fluency, and coherence:

Original text: "[ORIGINAL TEXT]"
Translated text: "[TRANSLATED TEXT]"

Provide a score from 0 to 100 to indicate the overall quality of the translation. There is no need for further explanation; simply return the score.


In [89]:
def generate_mapper(row):
  return {
        "[ORIGINAL LANGUAGE]": row["l1"],
        "[TRANSLATED LANGUAGE]": row["l2"],
        "[ORIGINAL TEXT]": row["src"],
        "[TRANSLATED TEXT]": row["mt"],
    }

In [90]:
def get_prompt(prompt_template: str, mapper: dict):
    prompt = prompt_template
    for tag, value in mapper.items():
        prompt = prompt.replace(tag, value)
    
    return prompt

In [91]:
df["prompt"] = df.apply(lambda row: get_prompt(prompt_template, generate_mapper(row)), axis=1)

In [92]:
print(df["prompt"][0])

Please evaluate the quality of the translation from en to zh based on accuracy, fluency, and coherence:

Original text: "Police said in a statement at the time that they made the decision to end the search for Jacsun with a "heavy heart," but noted they couldn't pinpoint a location in the landfill "to a point that would make continuing the search reasonable.""
Translated text: "警方在当时的一份声明中表示，他们决定以“沉重的心情”结束对 Jacsun 的搜索，但指出他们无法确定垃圾填埋场的位置，“以至于继续搜索是合理的”。"

Provide a score from 0 to 100 to indicate the overall quality of the translation. There is no need for further explanation; simply return the score.


In [93]:
from mistralai.client import MistralClient
from dotenv import load_dotenv
import os

load_dotenv()
MISTRAL_API = os.getenv("MISTRAL_API")
client = MistralClient(api_key=MISTRAL_API)

In [96]:
df["mistral_response"] = df["prompt"].map(lambda x: send_message_mistral(client, x, mistral_model="open-mistral-7b"))

In [123]:
for row in df[df["mistral_score"]>100].iterrows():
    idx = row[0]
    src = row[1]['lp']
    src = row[1]['src']
    mt = row[1]['mt']
    mistral_response = row[1]['mistral_response']

    print(idx)
    print(lp)
    print(src)
    print(mt)
    print(mistral_response)
    print()
    print()
    print()
    print()

1088
xh-zu
Mutta alastomien kuuluisuuksien sijaan he aikovat löytää seuraavan huippumallisukupolven 2017 NU Muses -kalenteriinsa.
But instead of naked squeezed out, they are going to find the next huippumallisukupolven 2017 Nude -kalenteriinsa Muses.
I cannot provide a score without further context as the translated text seems to contain errors and does not appear to accurately convey the meaning of the original text. The use of "naked squeezed out" and "huippamallisukupolven 2017 Nude -kalenteriinsa Muses" does not make sense in the given context. I would recommend consulting a professional translator for an accurate and high-quality translation.




1096
xh-zu
Vuonna 1987 Jumppanen pääsi Joensuun yliopistoon opiskelemaan tietojenkäsittelyä.
Year 1987 Jumppanen reach Estuary's university to study data processing.
I cannot provide a score without further explanation because the translation contains several errors. The translated text does not accurately convey the meaning of the origin

In [121]:
{
    1088: None,
    1096: 0,
    1195: None,
    2135: 50,
    2137: 95,
    2138: 50,
    2283:
    
    

    
}

In [103]:
def get_score(response: str):
    pattern = r"\d+(?:\.\d+)?"
    
    match = re.search(pattern, response)
    score = float(match.group(0)) if match else None

    return score

In [104]:
df["mistral_score"] = df["mistral_response"].map(get_score)

In [109]:
(df["mistral_score"]<0).sum()

0

In [69]:
df["mistral_response"].map(lambda x: type(x) is str).sum()

20

In [97]:
df.to_csv("mistral_response.csv", index=False)

In [98]:
df["mistral_response"]

0       I would give the translation a score of 100. B...
1       I would give the translation a score of 100. T...
2       I would give the translation a score of 100. T...
3       I would give a score of 100 for this translati...
4       Based on the provided text, I would give the t...
                              ...                        
2395    Based on the given text, I cannot provide a sc...
2396    Based on the provided text, I would give a sco...
2397    Based on the provided text, it is difficult to...
2398    Based on the given text, I would give a score ...
2399    Based on the given text, I cannot provide an a...
Name: mistral_response, Length: 2400, dtype: object